In [1]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c human-activity-recognition-har
!unzip /content/human-activity-recognition-har.zip

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          13670        284  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51          10327        213  1.0              
kapturovalexander/online-shop-2023                                  🏪🏬🛍️🛒 Online shop 2023                                3KB  2024-07-10 15:36:20            803         23  1.0              
ihelon/coffee-sales                     

In [2]:
import os
from glob import glob
from natsort import os_sorted
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
input_dir = "/content/HAR"
train_dir = input_dir + "/train"
test_dir  = input_dir + "/test"
df_paths  = glob(os.path.join(train_dir,'*','*.csv'))

In [4]:
concatdf = []
for activity_dir in os.listdir(train_dir):
    if activity_dir[0] == '.':
        continue
    print(activity_dir)
    for csv_file in os.listdir(os.path.join(train_dir,activity_dir)):
        print(os.path.join(train_dir, activity_dir, csv_file))
        df = pd.read_csv(os.path.join(train_dir, activity_dir, csv_file))
        df['label'] = activity_dir
        print(csv_file,df.shape[0],"->",200*(df.shape[0]//200))
        concatdf.append(df[:200*(df.shape[0]//200)])
        # break
    # break
frame = pd.concat(concatdf)

Standing
/content/HAR/train/Standing/24.csv
24.csv 544 -> 400
/content/HAR/train/Standing/21.csv
21.csv 2859 -> 2800
/content/HAR/train/Standing/12.csv
12.csv 1670 -> 1600
/content/HAR/train/Standing/10.csv
10.csv 1660 -> 1600
/content/HAR/train/Standing/19.csv
19.csv 2132 -> 2000
/content/HAR/train/Standing/30.csv
30.csv 3099 -> 3000
/content/HAR/train/Standing/5.csv
5.csv 1515 -> 1400
/content/HAR/train/Standing/35.csv
35.csv 1069 -> 1000
/content/HAR/train/Standing/6.csv
6.csv 709 -> 600
/content/HAR/train/Standing/18.csv
18.csv 1954 -> 1800
/content/HAR/train/Standing/13.csv
13.csv 1659 -> 1600
/content/HAR/train/Standing/20.csv
20.csv 5389 -> 5200
/content/HAR/train/Standing/7.csv
7.csv 2364 -> 2200
/content/HAR/train/Standing/16.csv
16.csv 1979 -> 1800
/content/HAR/train/Standing/27.csv
27.csv 1630 -> 1600
/content/HAR/train/Standing/28.csv
28.csv 1300 -> 1200
/content/HAR/train/Standing/8.csv
8.csv 3269 -> 3200
/content/HAR/train/Standing/3.csv
3.csv 2824 -> 2800
Jogging
/conten

In [5]:
frame

,x-axis,y-axis,z-axis,label
0,0.65,9.34,0.46,Standing
1,1.04,8.89,-1.04,Standing
2,1.50,8.96,-1.23,Standing
3,1.80,8.89,-0.65,Standing
4,1.61,8.92,-0.50,Standing
...,...,...,...,...
6795,-3.79,13.80,-8.50,Walking
6796,7.78,2.41,4.79,Walking
6797,-2.26,16.89,9.47,Walking
6798,-2.79,10.15,-1.92,Walking


In [6]:
nr = 200
new = []
frame = frame
for i in tqdm(range(len(frame)//nr)):
    x = frame[i*nr:(i+1)*nr]
    a = x.groupby(['label'])[['x-axis','y-axis','z-axis']].mean()
    a = a.rename(columns={'x-axis':'x-axismean','y-axis':'y-axismean','z-axis':'z-axismean'})
    b = x.groupby(['label'])[['x-axis','y-axis','z-axis']].std()
    b = b.rename(columns={'x-axis':'x-axisstd','y-axis':'y-axisstd','z-axis':'z-axisstd'})
    ab = a.join(b)
    new.append(ab)
train_df = pd.concat(new)
train_df

100%|██████████| 4345/4345 [00:49<00:00, 88.37it/s] 


,x-axismean,y-axismean,z-axismean,x-axisstd,y-axisstd,z-axisstd
label,,,,,,
Standing,1.96530,8.76485,-0.74930,0.837514,0.258985,0.722878
Standing,3.42905,8.37035,0.40510,0.674744,1.128058,0.491943
Standing,-1.13525,9.32545,1.31755,0.062926,0.037414,0.065796
Standing,-1.21850,9.33250,1.24245,0.045343,0.028982,0.055840
Standing,-1.18160,9.33415,1.34720,0.093950,0.119901,0.091251
...,...,...,...,...,...,...
Walking,0.51125,9.92445,0.53470,5.602911,5.526963,4.654065
Walking,0.61000,9.84335,0.95105,5.563308,5.408681,4.640613
Walking,0.38810,10.26210,0.55695,5.973234,5.402585,4.623317


In [7]:
train_df['activity'] = train_df.index
train_df

,x-axismean,y-axismean,z-axismean,x-axisstd,y-axisstd,z-axisstd,activity
label,,,,,,,
Standing,1.96530,8.76485,-0.74930,0.837514,0.258985,0.722878,Standing
Standing,3.42905,8.37035,0.40510,0.674744,1.128058,0.491943,Standing
Standing,-1.13525,9.32545,1.31755,0.062926,0.037414,0.065796,Standing
Standing,-1.21850,9.33250,1.24245,0.045343,0.028982,0.055840,Standing
Standing,-1.18160,9.33415,1.34720,0.093950,0.119901,0.091251,Standing
...,...,...,...,...,...,...,...
Walking,0.51125,9.92445,0.53470,5.602911,5.526963,4.654065,Walking
Walking,0.61000,9.84335,0.95105,5.563308,5.408681,4.640613,Walking
Walking,0.38810,10.26210,0.55695,5.973234,5.402585,4.623317,Walking


In [8]:
# Remove axis
train_df.reset_index(drop=True, inplace=True)
train_df

,x-axismean,y-axismean,z-axismean,x-axisstd,y-axisstd,z-axisstd,activity
0,1.96530,8.76485,-0.74930,0.837514,0.258985,0.722878,Standing
1,3.42905,8.37035,0.40510,0.674744,1.128058,0.491943,Standing
2,-1.13525,9.32545,1.31755,0.062926,0.037414,0.065796,Standing
3,-1.21850,9.33250,1.24245,0.045343,0.028982,0.055840,Standing
4,-1.18160,9.33415,1.34720,0.093950,0.119901,0.091251,Standing
...,...,...,...,...,...,...,...
4340,0.51125,9.92445,0.53470,5.602911,5.526963,4.654065,Walking
4341,0.61000,9.84335,0.95105,5.563308,5.408681,4.640613,Walking
4342,0.38810,10.26210,0.55695,5.973234,5.402585,4.623317,Walking
4343,0.49635,9.93440,0.58275,5.761783,5.293257,4.715050,Walking


<hr>

In [9]:
import lightgbm as lgb

In [10]:
X = train_df.drop(columns=['activity'])
y = train_df['activity']

In [11]:
lightmodel = lgb.LGBMClassifier()
lightmodel.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000446 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 4345, number of used features: 6
[LightGBM] [Info] Start training from score -2.439245
[LightGBM] [Info] Start training from score -1.158604
[LightGBM] [Info] Start training from score -2.996884
[LightGBM] [Info] Start training from score -3.189395
[LightGBM] [Info] Start training from score -2.200914
[LightGBM] [Info] Start training from score -0.923799
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

LGBMClassifier()

<hr>

In [12]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.3 MB/s eta 0:00:00


In [13]:
from catboost import CatBoostClassifier

In [14]:
catmodel = CatBoostClassifier()
catmodel.fit(X, y)

Learning rate set to 0.085283
0:	learn: 1.5684209	total: 76ms	remaining: 1m 15s
1:	learn: 1.4048091	total: 90.8ms	remaining: 45.3s
2:	learn: 1.2827609	total: 109ms	remaining: 36.2s
3:	learn: 1.1852095	total: 122ms	remaining: 30.5s
4:	learn: 1.1062449	total: 136ms	remaining: 27.1s
5:	learn: 1.0378994	total: 153ms	remaining: 25.4s
6:	learn: 0.9785672	total: 167ms	remaining: 23.8s
7:	learn: 0.9255093	total: 187ms	remaining: 23.2s
8:	learn: 0.8793218	total: 204ms	remaining: 22.5s
9:	learn: 0.8406227	total: 218ms	remaining: 21.6s
10:	learn: 0.8058712	total: 231ms	remaining: 20.8s
11:	learn: 0.7741940	total: 246ms	remaining: 20.3s
12:	learn: 0.7435775	total: 260ms	remaining: 19.8s
13:	learn: 0.7188376	total: 273ms	remaining: 19.3s
14:	learn: 0.6947200	total: 290ms	remaining: 19s
15:	learn: 0.6728515	total: 304ms	remaining: 18.7s
16:	learn: 0.6523699	total: 320ms	remaining: 18.5s
17:	learn: 0.6358089	total: 336ms	remaining: 18.3s
18:	learn: 0.6183277	total: 351ms	remaining: 18.1s
19:	learn: 0

<hr>

In [15]:
pip install xgboost


In [16]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [17]:
# Convert categorical labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Initialize XGBClassifier
xgmodel = XGBClassifier()

# Train the model
xgmodel.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

<hr>

# Ensemble model


In [25]:
submit_df = pd.read_csv("/content/sample_submission.csv")
folder_path = "/content/HAR/test"
for idx,row in submit_df[:].iterrows():
    csvfile = os.path.join(folder_path,row['id'])
    x = pd.read_csv(csvfile)
    x['label'] = 'xxx'
    a = x.groupby(['label'])[['x-axis','y-axis','z-axis']].mean()
    a = a.rename(columns={'x-axis':'x-axismean','y-axis':'y-axismean','z-axis':'z-axismean'})
    b = x.groupby(['label'])[['x-axis','y-axis','z-axis']].std()
    b = b.rename(columns={'x-axis':'x-axisstd','y-axis':'y-axisstd','z-axis':'z-axisstd'})
    ab = a.join(b)
    ab.reset_index(drop=True, inplace=True)

    lgb_pred = lightmodel.predict_proba(ab)
    cat_pred = catmodel.predict_proba(ab)
    xgb_pred = xgmodel.predict_proba(ab)

    # Average the predictions
    avg_pred = (lgb_pred + cat_pred + xgb_pred) / 3
    result = np.argmax(avg_pred, axis=1)
    # Convert numerical predictions back to original categorical labels
    result = label_encoder.inverse_transform(result)
    submit_df.at[idx,'class'] = result[0]
    # submit_df.at[idx,'class'] = label_encoder.inverse_transform(result)

In [26]:
import time
version = 'ensemble'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
submit_df.to_csv(csv_name,index=False)

v.ensemble.3022.csv
